In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from datetime import datetime
import time, random

# ---------------- CONFIG ----------------
CATEGORIES = ["mobiles", "laptops"]
PAGES_PER_CATEGORY = 10
PLATFORM = "Flipkart"


In [2]:
# ---------------- DRIVER ----------------
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [4]:
def safe_get_text(prod, selectors, default=""):
    """
    Tries multiple CSS selectors and returns the first matching element's text.
    If none found, returns default.
    """
    for selector in selectors:
        try:
            return prod.find_element(By.CSS_SELECTOR, selector).text
        except:
            continue
    return default


In [5]:
product_list = []

for category in CATEGORIES:
    print(f"Scraping category: {category}")
    
    for page in range(1, PAGES_PER_CATEGORY + 1):
        url = f"https://www.flipkart.com/search?q={category}&page={page}"
        driver.get(url)
        time.sleep(random.uniform(2,4))
        
        # Close login popup if present
        try:
            driver.find_element(By.XPATH, "//button[contains(text(),'✕')]").click()
        except:
            pass
        
        # Get product cards
        products = driver.find_elements(By.CSS_SELECTOR, "a.CGtC98")
        print(f"Found {len(products)} products on page {page}")
        
        for prod in products:
            try:
                # Product Name & Brand
                name = safe_get_text(prod, ["div.KzDlHZ", "div._4rR01T"])
                brand = name.split()[0] if name else ""
                
                # Current Price
                current_price_text = safe_get_text(prod, ["div.Nx9bqj._4b5DiR", "div._30jeq3"])
                current_price = float(current_price_text.replace("₹","").replace(",","")) if current_price_text else 0
                
                # Original Price
                original_price_text = safe_get_text(prod, ["div.yRaY8j.ZYYwLA", "div._3I9_wc"])
                original_price = float(original_price_text.replace("₹","").replace(",","")) if original_price_text else 0
                
                # Discount
                discount = safe_get_text(prod, ["div.UkUFwK span", "div._3Ay6Sb span"])
                
                # Rating
                rating_text = safe_get_text(prod, ["div.XQDdHH", "div._3LWZlK"])
                try:
                    rating = float(rating_text) if rating_text else 0
                except:
                    rating = 0
                
                # Reviews
                reviews = safe_get_text(prod, ["span.Wphh3N", "span._2_R_DZ"])
                
                # URL
                product_url = prod.get_attribute("href")
                
                scrape_date = datetime.now().strftime("%Y-%m-%d")
                
                product_list.append({
                    "Product_Name": name,
                    "Brand": brand,
                    "Current_Price": current_price,
                    "Original_Price": original_price,
                    "Discount(%)": discount,
                    "Rating": rating,
                    "Reviews_Count": reviews,
                    "Platform": PLATFORM,
                    "Category": category,
                    "Product_URL": product_url,
                    "Scrape_Date": scrape_date
                })
                
            except Exception as e:
                print("Error scraping product:", e)
                print("Product HTML:", prod.get_attribute("outerHTML")[:500])  # debug first 500 chars
                continue


Scraping category: mobiles
Found 24 products on page 1
Found 24 products on page 2
Found 24 products on page 3
Found 24 products on page 4
Found 24 products on page 5
Found 24 products on page 6
Found 24 products on page 7
Found 24 products on page 8
Found 24 products on page 9
Found 24 products on page 10
Scraping category: laptops
Found 24 products on page 1
Found 24 products on page 2
Found 24 products on page 3
Found 24 products on page 4
Found 24 products on page 5
Found 24 products on page 6
Found 24 products on page 7
Found 24 products on page 8
Found 24 products on page 9
Found 24 products on page 10


In [6]:
# Save to CSV
df = pd.DataFrame(product_list)
df.to_csv("flipkart_products.csv", index=False, encoding="utf-8-sig")
print(f" Scraping complete! Total products: {len(df)}. Saved to flipkart_products.csv")


 Scraping complete! Total products: 480. Saved to flipkart_products.csv


In [7]:
driver.quit()
